In [ ]:
from sympy.physics.units import *
from sympy import *

# Rounding:
import decimal
from decimal import Decimal as DX
def iso_round(obj, pv, rounding=decimal.ROUND_HALF_EVEN):
    import sympy
    """
    Rounding acc. to DIN EN ISO 80000-1:2013-08
    place value = Rundestellenwert
    """
    assert pv in set([
        # place value   #  round to:
        100,            #  3rd last digit before decimal
        10,             #  2nd last
        1,              #  last
        0.1,            #  1st digit after decimal
        0.01,           #  2nd
        0.001,          #  3rd
        0.0001,         #  4th
        0.00001,        #  5th
        0.000001,       #  6th
        0.0000001,      #  7th
        0.00000001,     #  8th
        0.000000001,    #  9th
        0.0000000001,   # 10th
        ])
    try:
        tmp = DX(str(float(obj)))
        obj = tmp.quantize(DX(str(pv)), rounding=rounding)
    except:
        for i in range(len(obj)):
            tmp = DX(str(float(obj[i])))
            obj[i] = tmp.quantize(DX(str(pv)), rounding=rounding)
    return obj

# LateX:
kwargs = {}
kwargs["mat_str"] = "bmatrix"
kwargs["mat_delim"] = ""
# kwargs["symbol_names"] = {FB: "F^{\mathsf B}", }

# Units:
(k, M, G ) = ( 10**3, 10**6, 10**9 )
(mm, cm, deg) = ( m/1000, m/100, pi/180)
Newton = kg*m/s**2
Pa     = Newton/m**2
MPa    = M*Pa
GPa    = G*Pa
kN     = k*Newton

# ---

x, a, q, EI, l = var("x, a, q, EI, l")

x = var("x")

w = (x*l - x*x)
W = a*w
Wp = diff(W,x)
Wpp = diff(Wp, x)

f1 = Wpp + q/2/EI*w
f2 = w

# Integrate:
I = integrate(f1*f2, x)

# Definite integral:
Idef = I.subs(x,l)

pprint("\nGalerkin = Equation (I) including definite integral:")
eq = Eq(Idef,0)
pprint(eq.simplify())

sol=solve(eq,a)
asol = sol[0]
pprint("\nSolution for a:")
pprint(asol)

# Substitute to get
Wsol = W.subs(a,asol)
pprint("\nSolution for w:")
pprint(Wsol)

# W(l/2)
pprint("\nw(l/2):")
Wl2=Wsol.subs(x,l/2)
pprint(Wl2)


pprint("\nPrinciple of Zero Virtual Power:")
a2, a3 = var("a2, a3")
W = (-a2*l -a3*l**2)*x
W += a2*x**2 + a3*x**3
Wp = diff(W, x)
Wpp = diff(Wp, x)
pprint("\nw:")
pprint(W)
# pprint(Wpp)

M = -EI*Wpp
Ui = integrate( -M*Wpp, (x,0,l) ) / 2
Ua = integrate( q*W, (x,0,l) )
pprint("\nInternal: Ui / EI:")
pprint((Ui/EI).simplify())
pprint("\nExternal: Ua:")
pprint(Ua.simplify())

# E = W - A
# pprint(E)
# eq = Eq(diff(E, a),0)
# sol = solve(eq, a)
# pprint(sol)


# Galerkin = Equation (I) including definite integral:
#      3     5
#   a⋅l     l ⋅q
# - ──── + ───── = 0
#    3     60⋅EI
#
# Solution for a:
#   2
#  l ⋅q
# ─────
# 20⋅EI
#
# Solution for w:
#  2   ⎛       2⎞
# l ⋅q⋅⎝l⋅x - x ⎠
# ───────────────
#      20⋅EI
#
# w(l/2):
#   4
#  l ⋅q
# ─────
# 80⋅EI
